<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-31 11:14:31
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20


-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  10.00 K
Current:  1.33 C
-------------------
Today PnL: -79.74 K (-0.59%)
Current PnL: -17.75 L (-12.37%)
CY Booked + Current PnL: -9.67 L (-6.74%)
-------------------
Total profit:  3.14 L
Total loss:  -20.90 L
-------------------
Total Booked + Current PnL: 17.31 L (14.85%)
Total Booked PnL: 35.06 L (30.08%)
Curr Year Booked PnL: 8.08 L (6.06%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 75.98 L (56.95%)
Deployed:  1.17 C
Current:  1.33 C
CAGR/XIRR %: 8.7%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction  Spread%   Current  \
92      VIPIND   488.80   489.00  -222.99       H-SC    13.60   88744.0   
17   BRITANNIA  5190.70  6446.05    16.13       H-LC     9.69  104211.0   
39    HINDALCO   651.95   761.55   -20.73       H-LC     5.26  109296.0   
4   APOLLOHOSP  6904.43  8285.00   -18.51       H-LC     6.54  111022.0   
74         SIS   477.00   477.00  2350.78       M-SC     6.63   65934.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
92      -6572.0   6611.0       -1.78         -6.89   7.45   0.04    161.0   
17      10778.0  11818.0        0.74         11.54  11.34  24.18     35.0   
39       4984.0  12547.0       -0.83          4.78  11.48  16.81     10.0   
4        7456.0  13256.0       -0.65          7.20  11.94  20.00     14.0   
74     -15156.0  15158.0        0.95        -18.69  22.99   0.00    238.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
92    -0.99        0.67    75.04    OX40N      NTT        MISC  
17     0.91        0.78    26.54     XY25      ATH        FMCG  
39     0.40        0.82    21.59      X5K      ATH      METALS  
4      0.56        0.83    22.29     X40N      BTT  HEALTHCARE  
74    -1.00        0.49    33.63    OX40N      NTT        MISC

In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost    FTT  Dev%_PE Conviction  Spread%  Current  Current P/L  \
62  QUESS   296.83  986.0   -34.99       M-SC    41.83  64769.0       -237.0   

     FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  RRR Ind  \
62  151164.0       -0.69         -0.36  233.39  232.18    196.0     -0.0   

    CurrAlloc%  Gained% Criteria Strategy Category  
62        0.49     0.63     XY24      NTT     MISC

In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction  Spread%   Current  \
22       CIPLA  1492.70  1795.0   -19.78       H-LC     1.70  211281.0   
10    BAJAJHFL   122.26   152.0   -10.01       H-LC     2.49  186976.0   
56        LTIM  5564.16  7230.2    -4.21       H-LC     2.68  184194.0   
5   ASIANPAINT  2961.56  4250.0   -17.94       H-LC     3.23  163091.0   
40  HINDUNILVR  2413.81  2723.0    -9.74       H-LC     4.24  282733.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
22       6781.0   34629.0       -1.10          3.32  16.39  20.25      7.0   
10     -13530.0   62300.0       -0.86         -6.75  33.32  24.33     30.0   
56     -16116.0   76091.0       -0.45         -8.05  41.31  29.94     26.0   
5      -38295.0  125906.0       -0.72        -19.02  77.20  43.51     20.0   
40      12386.0   22251.0        3.57          4.58   7.87  12.81     18.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
22     0.20        1.58    12.34     X40N      BTT   PHARMA  
10    -0.22        1.40     7.82     X40N      BTT  FINANCE  
56    -0.21        1.38    27.25     X200      ATH       IT  
5     -0.30        1.22    13.53      X40      BTT   PAINTS  
40     0.56        2.12    18.24     XY25      NTT     FMCG

In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction  Spread%   Current  \
40  HINDUNILVR  2413.81  2723.00    -9.74       H-LC     4.24  282733.0   
68  SAMMAANCAP   170.35   326.00  -167.05       M-SC     4.23   76944.0   
51      JIOFIN   310.58   387.00     7.61       H-LC    13.70  230650.0   
3   ALKYLAMINE  2347.98  4546.37     6.59       H-SC    19.68   99167.0   
88    UNITDSPR  1376.96  1644.00   -12.52       H-LC     7.45  176114.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
40      12386.0   22251.0        3.57          4.58    7.87  12.81     18.0   
68     -25266.0  118655.0        3.52        -24.72  154.21  91.37    210.0   
51      13244.0   40248.0        2.87          6.09   17.45  24.61     49.0   
3       -1796.0   96331.0        2.01         -1.78   97.14  93.63    147.0   
88      -5645.0   40894.0        1.40         -3.11   23.22  19.39     12.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
40     0.56        2.12    18.24     XY25      NTT       FMCG  
68    -0.21        0.58    24.91     XY25      NTT    FINANCE  
51     0.33        1.73    64.00     XY24      BTT    FINANCE  
3     -0.02        0.74    47.71       SR      ATH  CHEMICALS  
88    -0.14        1.32     4.16     X40N      NTT  BREWERIES

In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction  Spread%   Current  \
33  GREENPANEL   375.16   537.00   219.77       M-SC     8.41  142789.0   
64      RELAXO   902.64  1176.00   -35.12       H-SC    13.73   58630.0   
20        CAMS  3643.00  5226.82    -7.25       H-SC     0.30  104056.0   
18       BSOFT   628.85   836.99   -16.70       M-SC    12.46   69886.0   
75    SONACOMS   578.05  1006.00   -39.34       M-SC    13.77   78155.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
33     -40289.0  119272.0       -8.55        -22.01   83.53  43.14    234.0   
64     -56908.0   91897.0       -5.16        -49.25  156.74  30.28    133.0   
20       2052.0   42299.0       -4.47          2.01   40.65  43.48    122.0   
18     -40163.0   76588.0       -3.71        -36.50  109.59  33.10    216.0   
75     -23004.0   97897.0       -3.19        -22.74  125.26  74.03    198.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
33    -0.34        1.07    34.31     XY24      NTT      MISC  
64    -0.62        0.44    14.54     X40N      NTT  FOOTWEAR  
20     0.05        0.78    19.13     X40N      ATH      MISC  
18    -0.52        0.52    16.13       XR      ATH        IT  
75    -0.23        0.59    10.21       AR      BTT      AUTO

In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction  Spread%   Current  \
92      VIPIND   488.80   489.0  -222.99       H-SC    13.60   88744.0   
78    SYMPHONY  1306.42  1306.0   -31.08       M-SC     8.88  146864.0   
46  INDIGOPNTS  1407.73  1408.0   124.89       H-SC     6.17  147473.0   
86  UJJIVANSFB    52.77    53.0    36.52       M-SC    20.84  117990.0   
80   TATAELXSI  7332.28  7332.0   -19.66       H-MC     1.88   84287.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%  OTT%  CumlRnk  \
92      -6572.0   6611.0       -1.78         -6.89   7.45  0.04    161.0   
78     -24277.0  24218.0       -1.66        -14.19  16.49 -0.03    194.0   
46     -27086.0  27120.0       -1.12        -15.52  18.39  0.02    142.0   
86     -24489.0  25108.0        0.14        -17.19  21.28  0.44    249.0   
80     -18365.0  18358.0       -1.32        -17.89  21.78 -0.00    100.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
92    -0.99        0.67    75.04    OX40N      NTT      MISC  
78    -1.00        1.10     6.28    OX40N      NTT  DURABLES  
46    -1.00        1.11    27.88    OX40N      NTT    PAINTS  
86    -0.98        0.89    39.26    OX40N      NTT     BANKS  
80    -1.00        0.63    28.13    OX40N      NTT        IT

In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction  Spread%   Current  \
32  ENRIN  1377.95  3248.60      NaN        NaN     2.91   81215.0   
91    VBL   492.64   672.28    -8.22       H-LC    11.94  278415.0   
82    TCS  3794.03  4998.00   -26.74       H-LC    13.11  238181.0   
48   INFY  1461.46  2275.00   -15.29       H-LC     6.86  271901.0   
1     ABB  5551.76  7934.00   -39.57       H-LC     7.49  215806.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%  CumlRnk  \
32      46766.0       0.0        0.67        135.76   0.00  135.76      1.0   
91      13375.0   83274.0       -0.88          5.05  29.91   36.46      2.0   
82     -57753.0  151674.0       -0.08        -19.52  63.68   31.73      3.0   
48      10300.0  135325.0        0.35          3.94  49.77   55.67      5.0   
1        -713.0   93617.0       -0.37         -0.33  43.38   42.91      6.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
32      inf        0.61    24.19       AR      ATH  ELECTRICAL  
91     0.16        2.09    18.90     X40N      ATH        FMCG  
82    -0.38        1.79     0.00      X40      BTT          IT  
48     0.08        2.04    10.28      X40      BTT          IT  
1     -0.01        1.62    12.80       AR      NTT  ELECTRICAL

In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction  Spread%   Current  \
28  DIGITIDE   188.38   267.99      NaN        NaN     4.25   52341.0   
58   MEDANTA  1087.93  1486.00    19.04       H-SC    12.10  152644.0   
32     ENRIN  1377.95  3248.60      NaN        NaN     2.91   81215.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%  CumlRnk  \
28      11086.0   6349.0        0.73         26.87  12.13   42.26    197.0   
58      26444.0  19737.0        0.73         20.95  12.93   36.59    154.0   
32      46766.0      0.0        0.67        135.76   0.00  135.76      1.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
28     1.75        0.39    23.23     XY24      ATH          IT  
58     1.34        1.14    36.09     XY24      NTT  HEALTHCARE  
32      inf        0.61    24.19       AR      ATH  ELECTRICAL

In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction  Spread%   Current  \
53    KPIGREEN   497.21   731.64    25.50       H-SC    12.75  128268.0   
58     MEDANTA  1087.93  1486.00    19.04       H-SC    12.10  152644.0   
26       DABUR   505.20   735.00    -4.85       H-MC     4.92  206174.0   
20        CAMS  3643.00  5226.82    -7.25       H-SC     0.30  104056.0   
15  BLUESTARCO  1646.70  2326.38    -7.42       H-SC     9.77  173580.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
53       2971.0  56104.0       -1.12          2.37  43.74  47.15    143.0   
58      26444.0  19737.0        0.73         20.95  12.93  36.59    154.0   
26       9146.0  80470.0        1.27          4.64  39.03  45.49     96.0   
20       2052.0  42299.0       -4.47          2.01  40.65  43.48    122.0   
15       8910.0  59052.0       -1.08          5.41  34.02  41.28    119.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
53     0.05        0.96    60.47       MH      ATH       POWER  
58     1.34        1.14    36.09     XY24      NTT  HEALTHCARE  
26     0.11        1.55    18.29     XY24      BTT        FMCG  
20     0.05        0.78    19.13     X40N      ATH        MISC  
15     0.15        1.30    13.83     X40N      ATH          AC

In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction  Spread%   Current  \
32      ENRIN  1377.95  3248.60      NaN        NaN     2.91   81215.0   
94  WHIRLPOOL  1219.98  2270.00   -40.07       M-SC     7.50  100725.0   
45  INDIAMART  2327.09  4850.92   -47.14       H-SC    13.18  137758.0   
53   KPIGREEN   497.21   731.64    25.50       H-SC    12.75  128268.0   
95      WIPRO   243.46   420.00   -10.58       M-LC     5.56  153202.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%  CumlRnk  \
32      46766.0       0.0        0.67        135.76   0.00  135.76      1.0   
94       9226.0   69520.0       -2.96         10.08  69.02   86.07    205.0   
45      14422.0  119340.0        0.54         11.69  86.63  108.45    118.0   
53       2971.0   56104.0       -1.12          2.37  43.74   47.15    143.0   
95       2257.0  107195.0       -1.24          1.50  69.97   72.51     56.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
32      inf        0.61    24.19       AR      ATH  ELECTRICAL  
94     0.13        0.76    45.71       XR      NTT    DURABLES  
45     0.12        1.03    36.66       AR      ATH        MISC  
53     0.05        0.96    60.47       MH      ATH       POWER  
95     0.02        1.15     7.61       XR      NTT          IT

In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction  Spread%  Current  \
16  BLUSPRING   226.45    94.95      NaN        NaN     7.02  19933.0   
74        SIS   477.00   477.00  2350.78       M-SC     6.63  65934.0   
24  COFFEEDAY    59.14    80.00   -53.72       L-SC    20.99  67776.0   
32      ENRIN  1377.95  3248.60      NaN        NaN     2.91  81215.0   
92     VIPIND   488.80   489.00  -222.99       H-SC    13.60  88744.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
16     -29660.0    861.0       -0.69        -59.81    4.32  -58.07    204.0   
74     -15156.0  15158.0        0.95        -18.69   22.99    0.00    238.0   
24     -45773.0  85825.0       -2.49        -40.31  126.63   35.27    267.0   
32      46766.0      0.0        0.67        135.76    0.00  135.76      1.0   
92      -6572.0   6611.0       -1.78         -6.89    7.45    0.04    161.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
16   -34.45        0.15    33.68     XY24      ATH        MISC  
74    -1.00        0.49    33.63    OX40N      NTT        MISC  
24    -0.53        0.51    65.88       XR      NTT      HOTELS  
32      inf        0.61    24.19       AR      ATH  ELECTRICAL  
92    -0.99        0.67    75.04    OX40N      NTT        MISC

In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction  Spread%   Current  \
28   DIGITIDE   188.38   267.99      NaN        NaN     4.25   52341.0   
16  BLUSPRING   226.45    94.95      NaN        NaN     7.02   19933.0   
58    MEDANTA  1087.93  1486.00    19.04       H-SC    12.10  152644.0   
91        VBL   492.64   672.28    -8.22       H-LC    11.94  278415.0   
72     SHALBY   261.39   327.00   957.42       M-SC     9.08  143867.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
28      11086.0   6349.0        0.73         26.87  12.13  42.26    197.0   
16     -29660.0    861.0       -0.69        -59.81   4.32 -58.07    204.0   
58      26444.0  19737.0        0.73         20.95  12.93  36.59    154.0   
91      13375.0  83274.0       -0.88          5.05  29.91  36.46      2.0   
72     -37276.0  82738.0       -1.10        -20.58  57.51  25.10    228.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
28     1.75        0.39    23.23     XY24      ATH          IT  
16   -34.45        0.15    33.68     XY24      ATH        MISC  
58     1.34        1.14    36.09     XY24      NTT  HEALTHCARE  
91     0.16        2.09    18.90     X40N      ATH        FMCG  
72    -0.45        1.08    15.29     XY24      NTT  HEALTHCARE

In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       18.71
1     25       41.51
2     50       70.30

In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    41.37
LC    35.40
MC    22.04
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.59
X40N     14.75
X40      12.59
XY25     10.88
XR        8.98
OX40N     8.45
AR        8.24
X5K       2.21
MH        1.70
X200      1.38
SR        1.19
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.29
H-SC    25.11
H-MC    19.13
M-SC    14.86
M-LC     7.07
M-MC     2.57
L-SC     1.40
L-LC     1.04
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
FMCG                14.58         -0.65   34.33
IT                  11.72        -16.27   73.56
FINANCE              8.40        -16.63   58.95
BANKS                7.43        -15.15   62.13
MISC                 5.97        -20.77   77.16
PAINTS               5.71        -14.74   37.35
ELECTRICAL           5.44         -1.73   50.18
HEALTHCARE           4.61          1.97   24.06
AUTO                 4.33        -17.31   68.35
AC                   3.53         -2.70   37.47
INSURANCE            3.30         -6.98   40.94
METALS               2.95         -3.95   44.72
DURABLES             2.48        -10.04   33.89
FOOTWEAR             2.33        -32.36   76.10
TRAVEL               2.13        -39.66  105.76
CERAMICS             1.98        -28.40   70.22
STEEL                1.59         -5.87   84.76
PHARMA               1.58          3.21   16.39
CHEMICALS            1.40        -25.06  120.70
BREWERIES            1.32         -3.21   23.22
ENGINEERING          1.13        -27.41   79.76
JEWELLERY            1.12        -93.66  143.37
MINING               1.04         -8.09   36.14
CEMENT               0.98        -43.40  117.18
POWER                0.96          2.32   43.74
ENTERTAINMENT        0.89        -37.84  107.16
TEXTILES             0.55        -25.04   60.70
HOTELS               0.51        -67.54  126.63

In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      2768735.0     25
XR        1098275.0     13
AR        1048416.0     10
X40        773908.0     10
X40N       582768.0     12
XY25       487052.0      8
OX40N      345989.0     11
SR         184514.0      2
X5K        144358.0      2
MH          88069.0      2
X200        76091.0      1

In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
H-SC        2474163.0     26
M-SC        1646872.0     20
H-LC        1225051.0     20
H-MC        1154731.0     15
M-LC         436037.0      5
M-MC         333957.0      2
L-SC         212870.0      3
L-MC          56979.0      1
L-LC          50305.0      1

In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      967624.0      7
M-SC       XY24      793017.0      7
H-SC       AR        520823.0      3
H-LC       X40       506855.0      6
H-MC       XY24      446744.0      4
H-SC       XR        373454.0      4
M-MC       XY24      333957.0      2
M-SC       XR        288646.0      4
H-LC       X40N      277780.0      6
H-SC       X40N      225742.0      4
H-LC       AR        217037.0      2
H-MC       XY25      206503.0      2
           X40       192389.0      3
M-SC       AR        190161.0      2
H-SC       SR        184514.0      2
M-SC       OX40N     181729.0      5
M-LC       XY24      179935.0      2
L-SC       XR        151988.0      2
H-SC       OX40N     145902.0      5
M-LC       X5K       131811.0      1
H-MC       XR        120013.0      1
M-SC       XY25      118655.0      1
M-LC       XR        107195.0      1
H-LC       XY25       94493.0      3
H-MC       X40N       79246.0      2
H-LC       X200       76091.0      1
M-SC       X40        74664.0      1
L-SC       AR         60882.0      1
H-MC       AR         59513.0      1
L-MC       XR         56979.0      1
H-SC       MH         56104.0      1
L-LC       XY25       50305.0      1
H-LC       XY24       40248.0      1
H-MC       MH         31965.0      1
           OX40N      18358.0      1
M-LC       XY25       17096.0      1
H-LC       X5K        12547.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 49.0 seconds
